# Search Engine Results

In [35]:
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.snowball import DanishStemmer
from textblob import TextBlob
import lemmy
from sklearn.metrics import pairwise_kernels
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sentence_transformers import SentenceTransformer, util
import tensorflow as tf
import torch

embedder = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

In [36]:
dataset_name = '2020-02-26_2020-03-26_nodups_9k'

In [37]:
base = os.path.abspath('../')

def outname(name):
    return os.path.basename(dataset_name).split('.')[0] + name
    
outname_tfidf = outname('_distances_tfidf.csv')
outname_bert = outname('_encodings_bert.pt')
outname_df = outname('_preprocessed_df.csv')

print('Loading Processed Dataframe')
df = pd.read_csv(f'{base}/data/processed/{outname_df}', index_col=0)

print('Loading TFIDF distances')
%time tfidf = pd.read_csv( f'{base}/data/processed/{outname_tfidf}', index_col=0)

print('Loading BERT distances')
## It is not in Tensor format, but rather numpy array
# bert = pd.read_csv(f'{base}/data/processed/{outname_bert}', index_col=0)
%time bert = torch.load(f'{base}/data/processed/{outname_bert}')

print('\n All datasets loaded.')

Loading Processed Dataframe
Loading TFIDF distances
CPU times: user 1min 1s, sys: 3.24 s, total: 1min 5s
Wall time: 1min 9s
Loading BERT distances
CPU times: user 1.49 ms, sys: 29.8 ms, total: 31.3 ms
Wall time: 51.8 ms

 All datasets loaded.


In [38]:
tfidf = tfidf.reset_index()

In [39]:
## notice the corpus and title_processed
df.head(5)

,title,company,location,description,date,merged,corpus,title_processed
0,'Rundt om psykiatrien'; Unik mulighed for syge...,Psykiatrisk Center Ballerup,Ballerup,"Måske er du nyuddannet sygeplejerske, elle...",2020-03-02,'Rundt om psykiatrien'; Unik mulighed for syge...,'rund psykiatri unik mul sygeplejersk nysger p...,'rund psykiatri unik mul sygeplejersk nysger p...
1,(NY FRIST 23/3) Moos-Bjerre søger fire metodes...,Moos-Bjerre A/S,København,Moos-Bjerre søger i alt fire analytisk og ...,2020-03-03,(NY FRIST 23/3) Moos-Bjerre søger fire metodes...,ny frist 23/3 moos-bjer søg fir metodestærk pr...,ny frist 23/3 moos-bjer søg fir metodestærk pr...
2,(Senior) Numerical Specialist for design and i...,Ørsted Wind Power,Skærbæk near Fredericia or Gentofte,"Ørsted Wind Power, Skærbæk near Fredericia or ...",2020-03-13,(Senior) Numerical Specialist for design and i...,senior numerical specialist design and impleme...,senior numerical specialist design and impleme...
3,(Sr) Recruiting Partner,Biogen,København,Drive the recruiting and hiring process; H...,2020-02-27,(Sr) Recruiting Partner Biogen København D...,sr recruiting partn biog københavn driv the re...,sr recruiting partne
4,".NET Developer, Technical Consultant",Norriq Danmark A/S,Brøndby,Vores e-handelsteam udvider i både Aalborg...,2020-03-09,".NET Developer, Technical Consultant Norriq Da...",net develop technical consultant norriq danmar...,net develop technical consultant


In [40]:
def preprocess_text(text):
    # text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = str(text).lower().strip()

    # caveat: this might conflict with the english text
    da_stop_words = stopwords.words('danish')
    stemmer = DanishStemmer()
    lemmatizer = lemmy.load("da")

    # remove plurals
    textblob = TextBlob(text)
    singles = [stemmer.stem(word) for word in textblob.words]

    # remove danish stopwords
    no_stop_words = [word for word in singles if word not in da_stop_words]

    # join text so it can be lemmatized
    joined_text = " ".join(no_stop_words)

    # lemmatization
    final_text = lemmatizer.lemmatize("", joined_text)

    return final_text[0]

## SEARCH QUERY
---

In [58]:
search_query = 'Neurorehabilitering'
# search query - pædagog skov # Bert is best, but not great overall
# search_query = 'køkken i København' # slighly better than Jobindex
# search_query = 'læge neuropædiatri' # BERT exceeds
k = 20

## Direct custom search results

In [59]:
def find_direct_results(search_query):
    matching_entries = [df['title_processed'].index[df['title_processed'].str.contains(word, case=False)]
                        .values for word in search_query.split()]
    return list(set(matching_entries[0]).intersection(*matching_entries))

def print_direct_search_results(direct_results):
    agg_direct_results_indexes = []
    print('DIRECT RESULTS:', len(direct_results), '\n')
    for index, result in enumerate(direct_results):
        print(result, '-',  df['title'][result])
        agg_direct_results_indexes.append(result)
#     return agg_direct_results_indexes

direct_results = find_direct_results(preprocess_text(search_query))
print_direct_search_results(direct_results)

DIRECT RESULTS: 18 

7777 - Social- og sundhedsassistent til aftenvagt på Ringstedhave Neurorehabiliteringscenter
5317 - Neuropsykolog til Klinik for Højt Specialiseret Neurorehabilitering/Traumatisk Hjerneskade
5319 - Neuropsykologer til Neurorehabilitering - Kbh
9688 - Vikar for Sygeplejerske til S13, Neurorehabilitering, Regionshospitalet Skive, Hammel Neurocenter
8617 - Sygeplejerske søges til Center for Neurorehabilitering Næstved
8622 - Sygeplejerske søges til Neurorehabiliteringen, Svendborg
8623 - Sygeplejerske søges til Neurorehabiliteringen, Svendborg
7760 - Social- og sundhedsassistent til Neurorehabiliteringen i Grindsted
7761 - Social- og sundhedsassistent til Neurorehabiliteringen i Grindsted
2867 - Fysioterapeut til neurorehabilitering
2359 - Ergoterapeut , Afsnit for Neurorehabilitering, Frederikssund
1334 - Center for Neurorehabilitering(CfN) - Filadelfia søger SSA til aftenvagt
86 - 2 sygeplejersker søges til Neurorehabiliteringen i Grindsted
87 - 2 sygeplejersker søg

## TFIDF: Baseline model

In [60]:
%%time

def recommendations(data, results):
    sorted_results = []
    recommendation_indexes = []
    for result in results:
        sorted_distances = data[str(result)].sort_values().iteritems()
        
        for item_index, item_value in enumerate(sorted_distances):
            index, distance = item_value
            sorted_results.append({
                'item_index': index,
                'distance': distance,
                'title': df['title'][index]
            })
            
    agg_sorted_indexes = [x['item_index'] for x in sorted_results]
    ## removing direct_results for all results
    recommendation_indexes = [x for x in agg_sorted_indexes if x not in results]
        
    return recommendation_indexes

def display_results(data, name, direct_results):
    print (f'{name}\n')
    for index, recommendation in enumerate(recommendations(data=data, results=direct_results)[:k]):
        print(df['title'][recommendation])

def get_tfidf_recommendations(query):
    if len(direct_results) == 0:
        return None
    display_results(data=tfidf, name='TFIDF', direct_results=direct_results)

get_tfidf_recommendations(query=search_query)

TFIDF

Medarbejder til Center for Social Rehabilitering
Social- og sundhedsassistenter til aften- og nattevagt
Fysioterapeut til Rehabiliterings- og Plejecenter Skovbo
Social- og sundhedsassistent til fast aftenvagt i Ishøj Hjemmesygepleje
Medarbejder til Center for Social Rehabilitering
Social- og sundhedsassistenter til Bjerggårdshavens midlertidige døgnrehabilitering
Social- og sundhedsassistent til Center for Rehabilitering og Akutpleje
Social- og sundhedsassistent til fast aftenvagt
Social- og sundhedsassistent til Center for Rehabilitering og Akutpleje
Social- og sundhedsassistent i aftenvagt til Højdevang Sogns Plejehjem i Valby
Social- og Sundhedsassistent søges til dagvagt på Bakkehuset
Social- og sundhedsassistent til fast aftenvagt
Specialiseret børnerådgiver til ny Familieenhed i Veterancentret i Ringsted (genopslag)
Social- og sundhedsassistent i fast aftenvagt
9 centerterapeuter til rehabilitering på plejecentre
Social- og sundhedsassistent til aftenvagt på Margrethegårde

# BERT

In [61]:
%%time 
embedded_query = embedder.encode(search_query, convert_to_tensor=True)
sims = util.pytorch_cos_sim(embedded_query, bert)[0]

results = sorted(range(len(sims)), key=lambda x: sims[x], reverse=True)[:k]
## removing direct_results for all results
filtered_results = [x for x in results if x not in direct_results]


print (f'BERT\n')
for recommendation in filtered_results:
    print(df['title'][recommendation], '-', df['location'][recommendation]) 

BERT

neuropsykolog - Aarhus N
Overlæge i neuropædiatri - Aarhus N
Neurologisk fysioterapeut - barselsvikariat - Slangerup
Økonom til Neurocentrets centerledelse - København Ø
Reservelæge, Neurologi - Hillerød
Barselsvikariat Fysioterapeut Neuroteam - Brønderslev
Faglig velfunderet fysioterapeut søges til neurologisk team - Nykøbing M
Udviklingskonsulent til Specialområde Hjerneskade - Viby J
Erfaren neuropsykolog søges ved Neurologi, Aarhus Universitetshospital - Aarhus N
Lægesekretær til Neurokirurgisk Afdeling - Odense C
Lægesekretær til Neurokirurgisk Afdeling - Odense
Kunne du tænke dig at medvirke til udvikling af vores epilepsifunktion? - København Ø
Speciallæge til Kognitiv Indslusning og Stabiliserings Afsnit afdeling M, Psykiatrisk Center Sct. Hans - Roskilde
Spændende stilling som ergoterapeut på botilbud til borgere med erhvervet hjerneskade. - Mørke
CPU times: user 664 ms, sys: 20.5 ms, total: 685 ms
Wall time: 687 ms


# BERT: Bigger sized dataset
---